### Consigna del desafio 2
Objetivo del desafio:

El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

**1**. Crear sus propios vectores con Gensim basado en lo visto en clase con con el mismo dataset o con otro diferente.

**2**. Probar terminos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).

**3**. Graficarlos.

**4**. Obtener conclusiones.

### Import y configuraciones iniciales

In [3]:
import os
import platform
import multiprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.callbacks import CallbackAny2Vec

from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE

# reproducibilidad
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


### 1) Cargando el dataset

In [4]:
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip -O songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

# Ver artistas disponibles
os.listdir("./songs_dataset/")


--2025-09-14 18:27:10--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2025-09-14 18:27:11--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2075036 (2.0M) [application/zip]
Saving to: ‘songs_dataset.zip’

songs_dataset.zi

['drake.txt',
 'lil-wayne.txt',
 'missy-elliott.txt',
 'disney.txt',
 'leonard-cohen.txt',
 'bruce-springsteen.txt',
 'Lil_Wayne.txt',
 'notorious-big.txt',
 'bieber.txt',
 'blink-182.txt',
 'kanye.txt',
 'bjork.txt',
 'radiohead.txt',
 'nicki-minaj.txt',
 'bob-marley.txt',
 'bruno-mars.txt',
 'bob-dylan.txt',
 'beatles.txt',
 'al-green.txt',
 'alicia-keys.txt',
 'amy-winehouse.txt',
 'dj-khaled.txt',
 'r-kelly.txt',
 'eminem.txt',
 'adele.txt',
 'prince.txt',
 'patti-smith.txt',
 'paul-simon.txt',
 'michael-jackson.txt',
 'notorious_big.txt',
 'britney-spears.txt',
 'Kanye_West.txt',
 'dickinson.txt',
 'lin-manuel-miranda.txt',
 'lorde.txt',
 'lady-gaga.txt',
 'jimi-hendrix.txt',
 'nursery_rhymes.txt',
 'nirvana.txt',
 'johnny-cash.txt',
 'kanye-west.txt',
 'dr-seuss.txt',
 'ludacris.txt',
 'joni-mitchell.txt',
 'rihanna.txt',
 'cake.txt',
 'janisjoplin.txt',
 'dolly-parton.txt',
 'nickelback.txt']

In [5]:
# Cargar corpus de Justin Bieber
df = pd.read_csv('songs_dataset/bieber.txt', sep='/n', header=None, engine='python')
print("Cantidad de documentos:", df.shape[0])
df.head()

Cantidad de documentos: 3715


,0
0,What do you mean?
1,"Oh, oh, oh"
2,When you sometimes say yes
3,But you sometimes say no
4,What do you mean?


### 2) Preprocesamiento

In [6]:
sentence_tokens = []
for _, row in df.iterrows():
    if isinstance(row[0], str):
        tokens = simple_preprocess(row[0], deacc=True, min_len=2)
        sentence_tokens.append(tokens)

# ver ejemplo
sentence_tokens[:3]

[['what', 'do', 'you', 'mean'],
 ['oh', 'oh', 'oh'],
 ['when', 'you', 'sometimes', 'say', 'yes']]

### 2) Crear los vectores (word2vec)

In [7]:
# Callback para ver pérdida por época
class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
        self.loss_previous_step = 0.0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print(f'Loss after epoch {self.epoch}: {loss}')
        else:
            print(f'Loss after epoch {self.epoch}: {loss - self.loss_previous_step}')
        self.epoch += 1
        self.loss_previous_step = loss


In [8]:
# Crear modelo Word2Vec
workers = max(1, multiprocessing.cpu_count() - 1)

w2v_model = Word2Vec(min_count=2,       # palabra debe aparecer al menos 2 veces
                     window=5,          # contexto
                     vector_size=200,   # tamaño vector (100-300 típico)
                     negative=10,       # negative sampling
                     workers=workers,
                     sg=1,              # 1=Skip-gram, 0=CBOW
                     seed=RANDOM_SEED)

w2v_model.build_vocab(sentence_tokens)

print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
print("Cantidad de palabras distintas en el vocabulario:", len(w2v_model.wv.index_to_key))


Cantidad de docs en el corpus: 3715
Cantidad de palabras distintas en el vocabulario: 1145


### 3) Entrenar embeddings

In [9]:
# Entrenar embeddings
w2v_model.train(sentence_tokens,
                total_examples=w2v_model.corpus_count,
                epochs=30,
                compute_loss=True,
                callbacks=[EpochLogger()])


Loss after epoch 0: 97420.03125
Loss after epoch 1: 72218.765625
Loss after epoch 2: 69752.609375
Loss after epoch 3: 67938.65625
Loss after epoch 4: 66440.28125
Loss after epoch 5: 48608.28125
Loss after epoch 6: 61964.875
Loss after epoch 7: 60465.5625
Loss after epoch 8: 45984.375
Loss after epoch 9: 57349.25
Loss after epoch 10: 50188.375
Loss after epoch 11: 54250.0625
Loss after epoch 12: 54331.0
Loss after epoch 13: 51820.3125
Loss after epoch 14: 52691.6875
Loss after epoch 15: 51853.0625
Loss after epoch 16: 50519.875
Loss after epoch 17: 49527.3125
Loss after epoch 18: 48350.875
Loss after epoch 19: 37912.375
Loss after epoch 20: 46697.5
Loss after epoch 21: 46491.5
Loss after epoch 22: 40063.875
Loss after epoch 23: 36988.25
Loss after epoch 24: 46567.875
Loss after epoch 25: 36158.0
Loss after epoch 26: 35040.25
Loss after epoch 27: 46760.75
Loss after epoch 28: 46253.125
Loss after epoch 29: 44505.25


(518213, 833490)

### 4) Ensayar

In [10]:
# Ensayos con palabras de interés
terms = ["baby", "love", "girl", "heart", "money", "lonely"]

for t in terms:
    if t in w2v_model.wv.key_to_index:
        print(f"\nPalabras más relacionadas con '{t}':")
        for neigh, score in w2v_model.wv.most_similar(positive=[t], topn=8):
            print(f"   {neigh:15} {score:.3f}")
    else:
        print(f"\n'{t}' no está en el vocabulario.")



Palabras más relacionadas con 'baby':
   favorito        0.741
   begging         0.672
   teach           0.622
   compared        0.614
   tight           0.607
   sayin           0.600
   please          0.578
   fix             0.571

Palabras más relacionadas con 'love':
   madly           0.665
   choose          0.657
   swallow         0.655
   breathe         0.611
   romance         0.598
   afraid          0.598
   pill            0.596
   pulling         0.595

Palabras más relacionadas con 'girl':
   catching        0.670
   heartbreaker    0.646
   favorite        0.620
   cray            0.618
   ghetto          0.609
   laugh           0.588
   mhm             0.583
   plus            0.568

Palabras más relacionadas con 'heart':
   whose           0.641
   pieces          0.624
   deceiving       0.623
   breakin         0.617
   pound           0.604
   breaking        0.601
   met             0.596
   biggest         0.593

Palabras más relacionadas con 'money':
   

Conclusiones preliminares:
- El término "baby" se asocia con expresiones de súplica, ternura y afecto, lo cual refleja el uso típico en canciones de amor y pop juvenil donde "baby" aparece como apelativo romántico.

- El termino de “love” captura emociones intensas y referencias directas al romance.

- Para el termino "girl" se tiene un tono más juvenil y cotidiano, con asociaciones tanto afectivas (favorite, laugh) como negativas (heartbreaker).

- El termino "heart" está fuertemente ligado al dolor emocional y la ruptura (pieces, breaking, deceiving). El embedding refleja la narrativa frecuente en baladas románticas sobre corazones rotos y desilusiones amorosas.

- El termino “money” no se asocia tanto a riqueza material sino a cuestiones morales (lie, honestly, truth). Esto sugiere que en las letras de Bieber, el dinero aparece relacionado a la autenticidad y la sinceridad en las relaciones.

- El termino “Lonely” aparece vinculado a situaciones nocturnas y sociales (nights, faces).

### 5) Visualizar agrupacion de vectores

In [13]:
def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [15]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [16]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

Conclusiones:
- Los embeddings capturan bien el tono emocional de las letras de Bieber, donde predominan temas de amor, vulnerabilidad, rupturas y soledad.

- Se observan clusters semánticos claros:

    - Amor / cariño → (baby, love, girl)

    - Dolor / ruptura → (heart, lonely)

    - Honestidad / valores → (money)

Esto confirma que los embeddings entrenados con Gensim reflejan el contexto específico en el que cada palabra aparece en las canciones, diferente al sentido general que tendrían en otro corpus (ej. noticias, Wikipedia).
